## Cost of Capital

##### A notebook by **Matt Harrington (@conda_env)** for identifying optimal yield farm allocations.


#### Define portfolio constraints


In [1]:
# Constants
TAO = 5_000_000  # Total capital to be allocated
NETWORKS = ["mainnet", "polygon"]  # Networks to consider
TOKENS = ["WETH", "USDC"]  # Tokens to consider
SLIPPAGE = 0.001  # Slippage rate
MAX_VAULTS = 10  # Maximum number of vaults to allocate to
MAX_TVL_SHARE = 0.09  # Maximum share of TVL for allocation
# MIN_VAULT_TVL = TAO / MAX_VAULTS  # Minimum TVL for allocation
MIN_VAULT_TVL = 500_000 / MAX_TVL_SHARE  # Minimum TVL for allocation
MIN_APY = 15  # Minimum APY for allocation
HOLDING_DAYS = 30  # Number of days to hold the capital

# # Static API endpoint
# API_ENDPOINT = "https://api.vaults.fyi/v1/vaults?token=USDC&network=mainnet&tvl_min=1000000"

In [2]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from typing import Union, List
from pprint import pprint

load_dotenv()
BASE_URL = "https://api.vaults.fyi/v1/"
API_KEY = os.getenv("VAULTS_API_KEY")
API_KEY_HEADERS = {"X-API-Key": API_KEY}

/Users/matt/Dev/protocol-analysis/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


##### Fetch the data from the API and print vault data


In [3]:
# grab networks supported by vaults.fyi
networks_url = BASE_URL + "networks"

"""
curl -X 'GET' \
  'https://api.vaults.fyi/v1/networks' \
  -H 'accept: application/json' \
  -H 'x-api-key: a9cmW7qWgb8fIg4pMyvRaMgDIsFWWeSpOh6ljM7uuuI'
"""

networks_response = requests.get(networks_url, headers=API_KEY_HEADERS)
networks = networks_response.json()
pprint(networks)

[{'chainId': 42161, 'name': 'arbitrum'},
 {'chainId': 8453, 'name': 'base'},
 {'chainId': 100, 'name': 'gnosis'},
 {'chainId': 1, 'name': 'mainnet'},
 {'chainId': 10, 'name': 'optimism'},
 {'chainId': 137, 'name': 'polygon'}]


In [16]:
# grab vault metadata for the universe we're interested in
"""
curl -X 'GET' \
  'https://api.vaults.fyi/v1/detailed/vaults?token=USDC' \
  -H 'accept: application/json' \
  -H 'x-api-key: a9cmW7qWgb8fIg4pMyvRaMgDIsFWWeSpOh6ljM7uuuI'
"""
vaults_universe = []
vaults_url = BASE_URL + "detailed/vaults"
for token in TOKENS:
    for network in NETWORKS:
        try:
            vaults_params = {"token": token, "network": network}
            vaults_response = requests.get(
                vaults_url, headers=API_KEY_HEADERS, params=vaults_params
            )
            vaults = vaults_response.json()["data"]
            pprint(vaults)
            vaults_universe.extend(vaults)
        except Exception as e:
            print(f"Error fetching vaults for {token} on {network}")
            print(e)
params = {"token": "USDC", "network": "mainnet"}

[{'address': '0xd2eFB90C569eBD5b83D5cFB8632322edFAc203A5',
  'apy': {'1day': 128, '30day': 315, '7day': 120},
  'lendLink': None,
  'liquid': '0',
  'locked': '130557117054291966843',
  'name': 'Aave V3 WETH Lender',
  'network': 'mainnet',
  'numberOfHolders': 4,
  'rewards': [],
  'tags': [],
  'token': {'assetAddress': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
            'decimals': 18,
            'name': 'Wrapped Ether',
            'symbol': 'WETH'},
  'tvl': '130557117054291966843'},
 {'address': '0x030bA81f1c18d280636F32af80b9AAd02Cf0854e',
  'apy': {'1day': 31, '30day': 30, '7day': 33},
  'lendLink': 'https://app.aave.com/reserve-overview/?underlyingAsset=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&marketName=proto_mainnet',
  'liquid': '139652503187214959015720',
  'locked': '51290604688208657984930',
  'name': 'Aave v2 WETH',
  'network': 'mainnet',
  'numberOfHolders': 17036,
  'rewards': [],
  'tags': ['Lending'],
  'token': {'assetAddress': '0xC02aaA39b223FE8D0A0e5C4F

In [20]:
for i in range(len(vaults_universe)):
    if "apy" in vaults_universe[i]:
        for key, value in vaults_universe[i]["apy"].items():
            # expand token dict into columns
            vaults_universe[i][key] = [value]
            # drop the original token dict
        vaults_universe[i].pop("apy", None)
    if "token" in vaults_universe[i]:
        for key, value in vaults_universe[i]["token"].items():
            # expand token dict into columns
            vaults_universe[i][key] = [value]
            # drop the original token dict
        vaults_universe[i].pop("token", None)

# put that dict into a pandas dataframe
df = pd.DataFrame(vaults_universe)
df.head(2)
# df.info()

,name,address,network,tvl,liquid,locked,numberOfHolders,lendLink,tags,rewards,1day,7day,30day,assetAddress,symbol,decimals
0,[Wrapped Ether],0xd2eFB90C569eBD5b83D5cFB8632322edFAc203A5,mainnet,130557117054291966843,0,130557117054291966843,4.0,None,[],[],[128],[120],[315],[0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2],[WETH],[18]
1,[Wrapped Ether],0x030bA81f1c18d280636F32af80b9AAd02Cf0854e,mainnet,190943107875423617000650,139652503187214959015720,51290604688208657984930,17036.0,https://app.aave.com/reserve-overview/?underly...,[Lending],[],[31],[33],[30],[0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2],[WETH],[18]


In [26]:
# clean the dataframe
def extract_from_singleton(item):
    if isinstance(item, list) and len(item) == 1:
        return item[0]
    return item


# Apply function across DataFrame to clean up singleton list entries
df_cleaned = df.map(extract_from_singleton)


# Convert columns to appropriate data types using float to handle large numbers
df_cleaned["tvl"] = pd.to_numeric(df_cleaned["tvl"], errors="coerce")
df_cleaned["liquid"] = pd.to_numeric(df_cleaned["liquid"], errors="coerce")
df_cleaned["locked"] = pd.to_numeric(df_cleaned["locked"], errors="coerce")

# Ensuring the numberOfHolders and decimals columns are treated correctly
df_cleaned["numberOfHolders"] = pd.to_numeric(
    df_cleaned["numberOfHolders"], downcast="integer"
)
df_cleaned["decimals"] = pd.to_numeric(df_cleaned["decimals"], downcast="integer")

df_cleaned["name"] = df_cleaned["name"].astype(str)
df_cleaned["symbol"] = df_cleaned["symbol"].astype(str)
df_cleaned["network"] = df_cleaned["network"].astype(str)
df_cleaned["address"] = df_cleaned["address"].astype(str)

# if there are any Nan values for the APY's, set them to 0
df_cleaned["1day"] = df_cleaned["1day"].fillna(0)
df_cleaned["7day"] = df_cleaned["7day"].fillna(0)
df_cleaned["30day"] = df_cleaned["30day"].fillna(0)

df_cleaned.head(), df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             30 non-null     object 
 1   address          30 non-null     object 
 2   network          30 non-null     object 
 3   tvl              30 non-null     float64
 4   liquid           30 non-null     float64
 5   locked           30 non-null     float64
 6   numberOfHolders  11 non-null     float64
 7   lendLink         28 non-null     object 
 8   tags             30 non-null     object 
 9   rewards          30 non-null     object 
 10  1day             30 non-null     float64
 11  7day             30 non-null     float64
 12  30day            30 non-null     float64
 13  assetAddress     30 non-null     object 
 14  symbol           30 non-null     object 
 15  decimals         30 non-null     int8   
dtypes: float64(7), int8(1), object(8)
memory usage: 3.7+ KB


(            name                                     address  network  \
 0  Wrapped Ether  0xd2eFB90C569eBD5b83D5cFB8632322edFAc203A5  mainnet   
 1  Wrapped Ether  0x030bA81f1c18d280636F32af80b9AAd02Cf0854e  mainnet   
 2  Wrapped Ether  0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8  mainnet   
 3  Wrapped Ether  0x3750bc0283A73262ADc37043fD206306F168204C  mainnet   
 4  Wrapped Ether  0x706931Fb07348E174D17Cae3DA3EdBf9A26E8a88  mainnet   
 
             tvl        liquid        locked  numberOfHolders  \
 0  1.305571e+20  0.000000e+00  1.305571e+20              4.0   
 1  1.909431e+23  1.396525e+23  5.129060e+22          17036.0   
 2  7.224383e+23  1.649895e+23  5.574488e+23          11700.0   
 3  0.000000e+00  0.000000e+00  0.000000e+00              NaN   
 4  1.000594e+17  1.023260e+14  9.995706e+16              NaN   
 
                                             lendLink     tags rewards    1day  \
 0                                               None       []      []   128.0  

## Optimize the allocation of capital across the vaults


In [ ]:
# Number of vaults
num_vaults = len(cleaned_df)
vault = cleaned_df.copy()

### Use cvxpy.py to perform the convex optimization along the yield curves.


In [ ]:
import cvxpy as cp
import numpy as np

# Total Value Locked for each vault
tvls = np.array([vault["tvlInUsd"] for vault in vault_data])

# Optimization variables
allocations = cp.Variable(num_vaults)
selection = cp.Variable(num_vaults, boolean=True)

# Yield function for each vault (placeholder)
yields = cp.multiply(
    allocations, np.array([vault["apy"][YIELD_LOOKBACK] for vault in vault_data])
)

# Slippage adjusted yields
adjusted_yields = yields * (1 - SLIPPAGE)

# Objective: Maximize the total adjusted yield
objective = cp.Maximize(cp.sum(adjusted_yields))

# Constraints
constraints = [
    cp.sum(allocations) <= TAO,  # Total allocation must be TAO
    allocations
    <= selection * (0.5 * TAO),  # No more than half of TAO to any individual pool
    allocations >= 0,  # Allocations must be non-negative
    cp.sum(selection) <= MAX_VAULTS,  # No more than MAX_VAULTS vaults
    allocations <= tvls * MAX_TVL_SHARE,  # Allocation must be less than 10% of the TVL
]

# Problem
problem = cp.Problem(objective, constraints)

# Solve the problem using a solver that supports integer/binary programming
problem.solve()

# Check the status of the solution
if problem.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("Problem status:", problem.status)
else:
    # Optimal allocations
    optimal_allocations = allocations.value

allocated_vaults = np.where(optimal_allocations > 0)[0]

# Display the results
print("Optimal allocations:")
pprint(
    {
        f"{vault_data[i]['name']}": f"${optimal_allocations[i]/1_000:.2f}k"
        for i in allocated_vaults
    }
)

### Compose data & metadata into a single data table


In [ ]:
import pandas as pd

n_allocations = len(allocated_vaults)

# Convert allocations to a numpy array for easier handling
optimal_allocations_array = np.array(optimal_allocations)

# Calculate the anticipated APY for each allocated vault
post_deposit_apys = []
for i in allocated_vaults:
    tvl_v = vault_data[i]["tvlInUsd"]
    apy_v = vault_data[i]["apy"][YIELD_LOOKBACK] / 100
    allocation_v = optimal_allocations_array[i]
    adjusted_apy_v = apy_v * (tvl_v / (tvl_v + allocation_v))
    post_deposit_apys.append(adjusted_apy_v)

# Calculate the net APY for the portfolio
net_apy = np.average(
    [post_deposit_apys[i] / 100 for i in range(n_allocations)],
    weights=optimal_allocations_array[allocated_vaults],
)

# Calculate the expected USD return for each vault allocation
total_allocation = round(sum(optimal_allocations_array[allocated_vaults]))
print(f"Total allocation: {total_allocation}")
expected_yearly_usd_return = net_apy * total_allocation

# Prepare the data for the table
table_data = {
    "Protocol Name": [vault_data[i]["name"] for i in allocated_vaults],
    "Network": [vault_data[i]["network"] for i in allocated_vaults],
    "Current APY": [
        f'{vault_data[i]["apy"][YIELD_LOOKBACK] / 100}%' for i in allocated_vaults
    ],
    "Post-Entry APY*": [
        f"{round(post_deposit_apys[i], 1)}%" for i in range(n_allocations)
    ],
    "Asset Symbol": [vault_data[i]["token"]["symbol"] for i in allocated_vaults],
    "Vault Allocation": [
        f"${round(optimal_allocations_array[i] / 1_000, 2)}k" for i in allocated_vaults
    ],
    "Current TVL (USD)": [
        f'${round(vault_data[i]["tvlInUsd"] / 1_000_000, 2)}mm'
        for i in allocated_vaults
    ],
    "URL Link": [
        f"vaults.fyi/vaults/{vault_data[i]['network']}/{vault_data[i]['address']}"
        for i in allocated_vaults
    ],
    "Vault Address": [vault_data[i]["address"] for i in allocated_vaults],
}

# Create a DataFrame to display the table
vaults_table = pd.DataFrame(table_data)

# Print the expected USD return and net APY for the portfolio
print(
    f"Expected {HOLDING_DAYS}d USD return on the ${TAO / 1_000_000}mm "
    + f"portfolio: ${round(expected_yearly_usd_return * (HOLDING_DAYS / 365) / 1_000, 2)}k"
)
print(f"Net APY for the portfolio: {round(net_apy * 100, 2)}%")

total_allocation_str = (
    f"${total_allocation / 1_000:.2f}k"
    if total_allocation < 1_000_000
    else f"${total_allocation / 1_000_000:.2f}mm"
)

# Display the table
vaults_table

In [ ]:
# Assuming df is your DataFrame loaded with data
df = vaults_table

# Convert 'Vault Allocation' to numerical format and normalize to $100k units
df["Vault Allocation"] = (
    df["Vault Allocation"].replace({"\$": "", "k": "e3"}, regex=True).astype(float)
    / 100000
)

# Convert 'Current APY' to numerical format
df["Current APY"] = df["Current APY"].replace({"%": ""}, regex=True).astype(float)

## Plot the portfolio


In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.style.use("dark_background")
fig, ax1 = plt.subplots(figsize=(12, 7))

# Calculate bar positions
n = len(df)
index = range(n)
bar_width = 0.35

# Plot APY on primary y-axis
apy_bars = ax1.bar(index, df["Current APY"], bar_width, label="APY")
ax1.set_ylabel("APY (%)")
ax1.tick_params(axis="y")

# Annotate APY bars with the value above them
for bar in apy_bars:
    height = bar.get_height()
    ax1.annotate(
        f"{height:.2f}%",
        xy=(bar.get_x() + bar.get_width() / 2, height),
        xytext=(0, 3),  # 3 points vertical offset
        textcoords="offset points",
        ha="center",
        va="bottom",
        fontsize=11,
    )

# Create a second y-axis for Vault Allocation
ax2 = ax1.twinx()
vault_allocation_bars = ax2.bar(
    [p + bar_width for p in index],
    df["Vault Allocation"],
    bar_width,
    label="Vault Allocation",
    color="lightgreen",
)
ax2.set_ylabel("Vault Allocation")
ax2.tick_params(axis="y")

# Annotate Vault Allocation bars with the value above them
for bar in vault_allocation_bars:
    height = bar.get_height()
    ax2.annotate(
        f"${height*1e2:.2f}k",
        xy=(bar.get_x() + bar.get_width() / 2, height),
        xytext=(0, 3),  # 3 points vertical offset
        textcoords="offset points",
        ha="center",
        va="bottom",
        fontsize=11,
    )

# Add some text for labels, title, and custom x-axis tick labels, etc.
ax1.set_xlabel("Protocol")
ax1.set_title(
    f"APY & Vault Allocation by Protocol: {net_apy * 100:.2f}% Net APY on {total_allocation_str} Allocation"
)
ax1.set_xticks([p + bar_width / 2 for p in index])
# ax1.set_xticklabels(df['Protocol Name'], rotation=15, ha='center')
ax1.set_xticklabels(
    [f"{row['Protocol Name']} ({row['Network']})" for _, row in df.iterrows()],
    rotation=5,
    ha="center",
)

# Legend
handles, labels = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
fig.legend(handles + handles2, labels + labels2, loc="lower left", framealpha=0.25)

# Display the plot
plt.tight_layout()
plt.show()